# UR5 Simulation forward kinematics
**Author** Eduardo Gomez

Using the Denavit-Hartenberg parameters of the UR5, program the forward kinematics

In [15]:
# Import libraries
import zmqRemoteApi
import numpy as np
from time import sleep
from pathlib import Path
import pandas as pd
import utils
from utils import print_rounded_np

## Setup Data

Here we will set up the data for the Denavit-Hartenberg parameters. The data was from: ...

In [2]:
# First one I think is wrong



The matriz transformations can be expressed like these: 

$$
H_i^{i-1} = Rot_z(\theta_i)T(0,0,d_i)T(a_i, 0, 0)Rotx(\alpha_i)
$$

Esto se lee como una rotacion $\theta$ en $z$, una translacion $d$ en $z$, translacion $a$ en $x$ y rotacion $\alpha$ en $x$,

Here we'll define some utility functions

# Transformation matrix calcs

Because for the moment, all $\theta = 0$, the final coord is in the final matrix

## Setup simulation

To set up the simulation, open the `./sim/ur5-simulation.ttt` file. Then run the following code: 

> If there is an error, either the file is not opened or another script is controlling it

In [3]:
# Create a client to get connected to the zmqremoteApi sernver from CoppeliaSim
client = zmqRemoteApi.RemoteAPIClient()

# Get the remote object 'sim'
sim = client.getObject('sim')

print(sim)

<class 'zmqRemoteApi.sim'>


Here, we set up the id for controlling the robot

In [4]:
UR5_sim_obj_id = "/UR5"
base_id = sim.getObject(UR5_sim_obj_id)

# the last joint is the rotor for the end effector
# does not affect the final position
no_joints = 6
joint_sim_names = [None for _ in range(6)]

joint_sim_names[0] = f"{UR5_sim_obj_id}/joint"

for i in range(1,len(joint_sim_names)):
    link_joint_str = "/link/joint" * i
    joint_sim_names[i] = f"{UR5_sim_obj_id}{link_joint_str}"

connection_name = f"{UR5_sim_obj_id}/connection"

In [5]:
# Assert that names have id. Error is thrown if id doesnt exists
joint_id = [
    sim.getObject(name) for name in joint_sim_names
]
connection_id = sim.getObject(connection_name)

# Calculating Direct kinematics

Here we can use the direct kinematics using the matrices

In [8]:

matrices = utils.get_transformation_matrices_for_joint_list(sim,joint_id)
final = utils.mul_homogenous_matrixes(matrices)

assert len(matrices) + 1 == len(joint_id)
print(final)

p0 = sim.getObjectPosition(joint_id[0], sim.handle_world)
pf = sim.getObjectPosition(joint_id[-1], sim.handle_world)

print("initial")
utils.print_rounded_np(p0)

print("final")
utils.print_rounded_np(pf)

print("predicted")
p0_homogeneous =  utils.get_homogenous_point_3d(np.array(p0))
pp = final @ p0_homogeneous
utils.print_rounded_np(pp)




[[ 0.       0.      -1.      -0.09558]
 [ 0.       1.       0.       0.     ]
 [ 1.       0.       0.       0.97805]
 [ 0.       0.       0.       1.     ]]
initial
[ 0. -0.  0.]
final
[-0.0956  0.      0.9781]
predicted
[-0.0956 -0.      0.9781  1.    ]


# Try to get the angles and the translation

In [25]:

for prev_id, next_id in utils.window_iter(joint_id,2):
    m = np.array(sim.getObjectMatrix(next_id, prev_id)).reshape([3,4])
    # print_rounded_np(m)
    # print("x,y,z")
    # print_rounded_np(m[0:3,-1])
    
    rot_euler = np.array(sim.getObjectOrientation(next_id, prev_id))
    rot_yaw = sim.alphaBetaGammaToYawPitchRoll(*sim.getObjectOrientation(next_id, prev_id))

    print("euler")
    print_rounded_np(rot_euler)
    print("rotation")
    print_rounded_np(rot_yaw)

# No encuentro la manera de hacer una rotacino en y con solamente z y x











euler
[ 0.     -1.5708  0.    ]
rotation
[ 0.     -1.5708  0.    ]
euler
[-0.  0. -0.]
rotation
[-0. -0. -0.]
euler
[-0.  0. -0.]
rotation
[-0. -0. -0.]
euler
[0.     1.5708 0.    ]
rotation
[0.     1.5708 0.    ]
euler
[ 0.     -1.5708  0.    ]
rotation
[ 0.     -1.5708  0.    ]


In [44]:

# how to build up rotation matrixes

rot_y, _ = utils.matrices.rotation_3d_deg(yau_z=0,theta_y=90,phi_x=0)

possible_angles = [90,-90, 45, -45]

no_rot = utils.NO_TRANSLATION_VEC

for z in possible_angles:
    for x in possible_angles:
        rot_zx, _ = utils.matrices.rotation_3d_deg(yau_z=z,theta_y=0,phi_x=x)
        v1 = utils.add_translation_to_rotation_3d(rot_y,np.array([0,0,0]))
        v2 = utils.add_translation_to_rotation_3d(rotation_3d=rot_zx, p=no_rot)
        
        print(f"for z={z} x={x}")
        print_rounded_np(v1)
        print_rounded_np(v2)
        print()
        
        
        
        










for z=90 x=90
[[ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [-1.  0.  0.  0.]
 [ 0.  0.  0.  1.]]
[[ 0. -0.  1.  0.]
 [ 1.  0. -0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]]

for z=90 x=-90
[[ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [-1.  0.  0.  0.]
 [ 0.  0.  0.  1.]]
[[ 0. -0. -1.  0.]
 [ 1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  0.  1.]]

for z=90 x=45
[[ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [-1.  0.  0.  0.]
 [ 0.  0.  0.  1.]]
[[ 0.     -0.7071  0.7071  0.    ]
 [ 1.      0.     -0.      0.    ]
 [ 0.      0.7071  0.7071  0.    ]
 [ 0.      0.      0.      1.    ]]

for z=90 x=-45
[[ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [-1.  0.  0.  0.]
 [ 0.  0.  0.  1.]]
[[ 0.     -0.7071 -0.7071  0.    ]
 [ 1.      0.      0.      0.    ]
 [ 0.     -0.7071  0.7071  0.    ]
 [ 0.      0.      0.      1.    ]]

for z=-90 x=90
[[ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]
 [-1.  0.  0.  0.]
 [ 0.  0.  0.  1.]]
[[ 0.  0. -1.  0.]
 [-1.  0. -0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]]

for z=-90 x=-90
[[ 0.  0

Here we stop the simulation

In [9]:
sim.stopSimulation()

0